In [13]:
!pip install --upgrade langchain_core langchain_community -qqq
!pip install --upgrade vllm -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [11]:
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer

In [10]:
pip install langchain_chroma sentence_transformers chromadb

In [8]:
import chromadb

In [1]:
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer

revision = None  # Replace with the specific revision to ensure reproducibility if the model is updated.

model = SentenceTransformer("avsolatorio/GIST-large-Embedding-v0", revision=revision)

texts = [
    "Illustration of the REaLTabFormer model. The left block shows the non-relational tabular data model using GPT-2 with a causal LM head. In contrast, the right block shows how a relational dataset's child table is modeled using a sequence-to-sequence (Seq2Seq) model. The Seq2Seq model uses the observations in the parent table to condition the generation of the observations in the child table. The trained GPT-2 model on the parent table, with weights frozen, is also used as the encoder in the Seq2Seq model.",
    "Predicting human mobility holds significant practical value, with applications ranging from enhancing disaster risk planning to simulating epidemic spread. In this paper, we present the GeoFormer, a decoder-only transformer model adapted from the GPT architecture to forecast human mobility.",
    "As the economies of Southeast Asia continue adopting digital technologies, policy makers increasingly ask how to prepare the workforce for emerging labor demands. However, little is known about the skills that workers need to adapt to these changes"
]

# Compute embeddings
embeddings = model.encode(texts, convert_to_tensor=True)

# Compute cosine-similarity for each pair of sentences
scores = F.cosine_similarity(embeddings.unsqueeze(1), embeddings.unsqueeze(0), dim=-1)

print(scores.cpu().numpy())


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[[1.         0.49539524 0.31134686]
 [0.4953953  1.         0.40336773]
 [0.31134686 0.40336773 1.        ]]


In [14]:
from langchain_community.llms import VLLM

model_id = "TitanML/ChatQA-1.5-8B-AWQ-4bit"
llm = VLLM(
    model=model_id,
    trust_remote_code=True,
    max_new_tokens=128,
    top_k=10,
    top_p=0.95,
    temperature=0.8,
    vllm_kwargs={"quantization": "awq"},
)

print(llm.invoke("What is the capital of France ?"))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/960 [00:00<?, ?B/s]

WARNING 06-09 15:31:15 config.py:213] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 06-09 15:31:15 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='TitanML/ChatQA-1.5-8B-AWQ-4bit', speculative_config=None, tokenizer='TitanML/ChatQA-1.5-8B-AWQ-4bit', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=TitanML/ChatQA-1.5-8B-AWQ-4bit)


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.68G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/63.5k [00:00<?, ?B/s]

INFO 06-09 15:34:59 model_runner.py:146] Loading model weights took 5.3441 GB
INFO 06-09 15:35:05 gpu_executor.py:83] # GPU blocks: 2807, # CPU blocks: 2048
INFO 06-09 15:35:08 model_runner.py:854] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-09 15:35:08 model_runner.py:858] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 06-09 15:35:30 model_runner.py:924] Graph capturing finished in 22 secs.


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.23it/s, Generation Speed: 3.27 toks/s]

In [ ]:
from langchain_core.prompts import PromptTemplate
system = "System: This is a chat between a user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions based on the context. The assistant should also indicate when the answer cannot be found in the context."
instruction = "Please give a full and complete answer for the question."
template = "<|begin_of_text|>" + system + "\n\n" + "{context}" + "\n\n" + "User: " + instruction + " " + "{question}"
prompt = PromptTemplate.from_template(template)

In [ ]:
qa =  {"contex" : """NVIDIA (NASDAQ: NVDA) today reported revenue for the fourth quarter ended January 28, 2024, of $22.1 billion, up 22% from the previous quarter and up 265% from a year ago.\nFor the quarter, GAAP earnings per diluted share was $4.93, up 33% from the previous quarter and up 765% from a year ago. Non-GAAP earnings per diluted share was $5.16, up 28% from the previous quarter and up 486% from a year ago.\nQ4 Fiscal 2024 Summary\nGAAP\n| $ in millions, except earnings per share | Q4 FY24 | Q3 FY24 | Q4 FY23 | Q/Q | Y/Y |\n| Revenue | $22,103 | $18,120 | $6,051 | Up 22% | Up 265% |\n| Gross margin | 76.0% | 74.0% | 63.3% | Up 2.0 pts | Up 12.7 pts |\n| Operating expenses | $3,176 | $2,983 | $2,576 | Up 6% | Up 23% |\n| Operating income | $13,615 | $10,417 | $1,257 | Up 31% | Up 983% |\n| Net income | $12,285 | $9,243 | $1,414 | Up 33% | Up 769% |\n| Diluted earnings per share | $4.93 | $3.71 | $0.57 | Up 33% | Up 765% |""",
"question" : "what is the percentage change of the net income from Q4 FY23 to Q4 FY24?"}
prompt.invoke(qa)

In [ ]:
(prompt | llm).inv

In [20]:
chroma_client = chromadb.Client()

In [15]:
from sentence_transformers import SentenceTransformer
import chromadb

class CustomEmbeddingFunction:
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)

    def __call__(self, input):
        return self.model.encode(input)

# Define the custom embedding function using the pre-trained model
embedding_function = CustomEmbeddingFunction(model_name="avsolatorio/GIST-large-Embedding-v0")

# Initialize the Chroma client
chroma_client = chromadb.Client()

# Create a collection with the custom embedding function
collection = chroma_client.create_collection(name="embeddings", embedding_function=embedding_function)

# Your vector store creation can proceed here as well
# vector_store = Chroma(client=client, collection_name="pdf_embeddings", embedding_function=embedding_function)


In [13]:
import chromadb

# Initialize the Chroma client
chroma_client = chromadb.Client()

# Delete the existing collection if it exists
collection_name = "pdf_embeddings"
if collection_name in chroma_client.list_collections():
    chroma_client.delete_collection(name=collection_name)

# Now you can create the collection again



In [ ]:
docs = [doc.page_content for doc in documents]

In [ ]:
%%time
collection.add(
    documents=docs,
    ids=[str(i) for i in range(len(docs))]
)

In [ ]:
from langchain_chroma import Chroma
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)


langchain_chroma = Chroma(
    client=chroma_client,
    collection_name="embeddings",
    embedding_function=embedding_function,
)
retriever = langchain_chroma.as_retriever()
langchain_chroma._collection.count()

In [ ]:
rag_chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)